### `gold_posicao_atual` – Posição em Tempo Real da Frota  
**~6.500 linhas | ~5 segundos | Atualiza a cada 10 min**

| Etapa | O que faz | Resultado |
|------|----------|----------|
| 1. Lê silver | `silver_posicao` + `silver_linhas` | Dados limpos e enriquecidos |
| 2. Enriquecimento | Join com `letreiro`, `nome_oficial`, `route_id` | Informações oficiais da linha |
| 3. Fallback seguro | `coalesce()` no letreiro | Nunca dá nulo |
| 4. Última posição | `row_number()` + `desc(ts_coleta)` | **Só 1 linha por ônibus** |
| 5. Salva Delta | `overwrite` + `overwriteSchema` | Tabela leve e rápida |
| 6. Pronto pro mapa | `latitude`, `longitude`, `prefixo` | **Mapa ao vivo com 6.500 pontos** |

**Tabela final:** `workspace.sptrans.gold_posicao_atual`  
**Uso:** Dashboard, Power BI, API, mapa interativo  
**Linha mágica:** `.filter(col("rn") == 1)` → reduz de milhões pra 6.500!

**Essa tabela agora responde perguntas como:**

-  Onde está cada ônibus agora?
-  Qual linha ele opera?
-  Qual o nome oficial da rota?
-  É acessível?
-  Último horário coletado?

In [0]:
%sql
SELECT COUNT(*) AS total_registros
FROM workspace.sptrans.gold_posicao_atual;

total_registros
4729


Codigo atualizado em 10.11 - Avisar Ana
Salva SÓ A ÚLTIMA posição de cada ônibus
Tabela fica com ~6.500 linhas (leve e rápida)

In [0]:
# ==============================================================
# GOLD_POSICAO_ATUAL – VERSÃO FINAL (10/11/2025)
# Só a última posição de cada ônibus → ~6.500 linhas no máximo
# ==============================================================

from pyspark.sql.functions import col, coalesce, desc, row_number
from pyspark.sql.window import Window

# --------------------------------------------------------------
# 1. Leitura das tabelas Silver (já estão funcionando perfeitamente)
# --------------------------------------------------------------
silver_pos = spark.table("workspace.sptrans.silver_posicao")
silver_lin = spark.table("workspace.sptrans.silver_linhas")

# --------------------------------------------------------------
# 2. Enriquecimento com informações da linha (letreiro, nome oficial, GTFS)
# --------------------------------------------------------------
enriched = (
    silver_pos.alias("p")
    .join(silver_lin.alias("l"), col("p.cod_linha") == col("l.cod_linha"), "left")
    .select(
        col("p.prefixo"),
        col("p.cod_linha"),
        # Se não achar na silver_linhas, usa o letreiro que veio da posição
        coalesce(col("l.letreiro"), col("p.letreiro")).alias("letreiro"),
        col("l.nome_oficial"),
        col("l.route_id"),
        col("l.agency_id"),
        col("p.lat").alias("latitude"),
        col("p.lng").alias("longitude"),
        col("p.sentido"),
        col("p.acessivel"),
        col("p.ts_coleta")
    )
)

# --------------------------------------------------------------
# 3. Mantém APENAS a última posição de cada ônibus
# --------------------------------------------------------------
window = Window.partitionBy("prefixo").orderBy(desc("ts_coleta"))

gold_posicao_atual = (
    enriched
    .withColumn("rn", row_number().over(window))
    .filter(col("rn") == 1)   # ← AQUI ESTÁ A MÁGICA
    .drop("rn")
    .select(
        "prefixo",
        "cod_linha",
        "letreiro",
        "nome_oficial",
        "route_id",
        "agency_id",
        "latitude",
        "longitude",
        "sentido",
        "acessivel",
        "ts_coleta"
    )
)

# --------------------------------------------------------------
# 4. Salva sobrescrevendo a tabela (agora só 6.500 linhas!)
# --------------------------------------------------------------
(gold_posicao_atual.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("workspace.sptrans.gold_posicao_atual")
)

# --------------------------------------------------------------
# 5. Resultado
# --------------------------------------------------------------
qtd_onibus = gold_posicao_atual.count()

print(f"POSIÇÃO ATUAL ATUALIZADA! {qtd_onibus} ônibus ativos na frota agora.")
print("Tabela: workspace.sptrans.gold_posicao_atual")
print("Pronto para mapa ao vivo no Power BI / Databricks SQL")

# Visualização limpa (nunca mais erro de display)
display(gold_posicao_atual.orderBy(desc("ts_coleta")).limit(30))

POSIÇÃO ATUAL ATUALIZADA! 4729 ônibus ativos na frota agora.
Tabela: workspace.sptrans.gold_posicao_atual
Pronto para mapa ao vivo no Power BI / Databricks SQL


prefixo,cod_linha,letreiro,nome_oficial,route_id,agency_id,latitude,longitude,sentido,acessivel,ts_coleta
21730,33432,172N,null,null,null,-23.50610925,-46.603337249999996,2,true,2025-11-09T23:38:19.789Z
10428,1251,8001,null,null,null,-23.5196005,-46.700601,1,true,2025-11-09T23:38:19.789Z
32008,375,2583,null,null,null,-23.504152499999996,-46.432775,1,true,2025-11-09T23:38:19.789Z
82626,34856,9050,null,null,null,-23.5353895,-46.704792499999996,2,true,2025-11-09T23:38:19.789Z
64130,34458,574A,null,null,null,-23.646586749999997,-46.645936000000006,2,true,2025-11-09T23:38:19.789Z
47720,33809,3741,null,null,null,-23.56229,-46.4560265,2,true,2025-11-09T23:38:19.789Z
64465,262,476L,null,null,null,-23.589351999999998,-46.633652,1,true,2025-11-09T23:38:19.789Z
64265,2242,609J,null,null,null,-23.6223755,-46.66169549999999,1,true,2025-11-09T23:38:19.789Z
61567,34211,6960,null,null,null,-23.7113265,-46.71106575,2,true,2025-11-09T23:38:19.789Z
16345,34779,8013,null,null,null,-23.434838,-46.786431125,2,true,2025-11-09T23:38:19.789Z


A SPTrans realmente opera veículos que não são ônibus de linha regular, por isso deixá-los na mesma tabela pode confundir análises de rota, embarque e KPIs de frota.